In [ ]:
!nvcc --version

In [13]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-04-19 12:31:51--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.40.40.50, 23.40.40.83
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.40.40.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run.1’

cuda_11.8.0_520.61. 100%[===================>]   4.04G  78.7MB/s    in 55s     

2025-04-19 12:32:47 (75.0 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run.1’ saved [4336730777/4336730777]



In [14]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


# Q1

In [15]:
%%writefile q1.cu
// %%writefile q1.cu
#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>

__global__ void vecAddKernel(int *A, int *B, int *C,int n){
    int i=blockDim.x*blockIdx.x+threadIdx.x;

    printf("In Kernel function, i=%d ",i);
    if(i<n){
        C[i]=A[i]+B[i];
    }
}

int main(){
    int n;
    printf("Enter Number of Elements: ");
    scanf("%d",&n);

    int *a,*b,*c;

    a=(int *)malloc(n*sizeof(int));
    b=(int *)malloc(n*sizeof(int));
    c=(int *)malloc(n*sizeof(int));

    printf("\n Enter Array A: ");

    for(int i=0;i<n;i++){
        scanf("%d", &a[i]);
    }

    printf("\n Enter Array B: ");

    for(int i=0;i<n;i++){
        scanf("%d", &b[i]);
    }

    int *da, *db, *dc;

    int size=n*sizeof(int);

    //part-1
    cudaMalloc((void **)&da,size);
    cudaMalloc((void **)&db,size);
    cudaMalloc((void **)&dc,size);

    cudaMemcpy(da, a, size,cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, size,cudaMemcpyHostToDevice);

    // part-2 blocks and threads
    // int blocks = (n + 255) / 256;
    // vecAddKernel<<<blocks, 256>>>(da, db, dc, n);
    // OR
    vecAddKernel<<< ceil(n/256.0), 256 >>>(da, db, dc, n);

    // Error check
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
    }

    // part-3 transfer to C

    cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

    printf("\nFinal Array after adding both is : ");

    for(int i=0;i<n;i++){
        printf("%d\t",c[i]);
    }

    free(a);
    free(b);
    free(c);
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
}

Overwriting q1.cu


In [16]:
!nvcc q1.cu -o q1.out

In [17]:
!./q1.out

Enter Number of Elements: 3

 Enter Array A: 2 4 6

 Enter Array B: 1 2 43
In Kernel function, i=128 In Kernel function, i=129 In Kernel function, i=130 In Kernel function, i=131 In Kernel function, i=132 In Kernel function, i=133 In Kernel function, i=134 In Kernel function, i=135 In Kernel function, i=136 In Kernel function, i=137 In Kernel function, i=138 In Kernel function, i=139 In Kernel function, i=140 In Kernel function, i=141 In Kernel function, i=142 In Kernel function, i=143 In Kernel function, i=144 In Kernel function, i=145 In Kernel function, i=146 In Kernel function, i=147 In Kernel function, i=148 In Kernel function, i=149 In Kernel function, i=150 In Kernel function, i=151 In Kernel function, i=152 In Kernel function, i=153 In Kernel function, i=154 In Kernel function, i=155 In Kernel function, i=156 In Kernel function, i=157 In Kernel function, i=158 In Kernel function, i=159 In Kernel function, i=192 In Kernel function, i=193 In Kernel function, i=194 In Kernel funct

In [20]:
%%writefile q1_a.cu
#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>

__global__ void vecAddKernel(int *A, int *B, int *C,int n){
    int i=blockDim.x*blockIdx.x+threadIdx.x;

    printf("In Kernel function, i=%d ",i);
    if(i<n){
        C[i]=A[i]+B[i];
    }
}

int main(){
    int n;
    printf("Enter Number of Elements: ");
    scanf("%d",&n);

    int *a,*b,*c;

    a=(int *)malloc(n*sizeof(int));
    b=(int *)malloc(n*sizeof(int));
    c=(int *)malloc(n*sizeof(int));

    printf("\n Enter Array A: ");

    for(int i=0;i<n;i++){
        scanf("%d", &a[i]);
    }

    printf("\n Enter Array B: ");

    for(int i=0;i<n;i++){
        scanf("%d", &b[i]);
    }

    int *da, *db, *dc;

    int size=n*sizeof(int);

    //part-1
    cudaMalloc((void **)&da,size);
    cudaMalloc((void **)&db,size);
    cudaMalloc((void **)&dc,size);

    cudaMemcpy(da, a, size,cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, size,cudaMemcpyHostToDevice);

    // part-2 blocks and threads
    // int blocks = (n + 255) / 256;
    // vecAddKernel<<<blocks, 256>>>(da, db, dc, n);
    // OR
    // vecAddKernel<<< ceil(n/256.0), 256 >>>(da, db, dc, n);
    // OR
    vecAddKernel<<<1,n>>>(da,db,dc,n);

    // Error check
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
    }

    // part-3 transfer to C

    cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

    printf("\nFinal Array after adding both is : ");

    for(int i=0;i<n;i++){
        printf("%d\t",c[i]);
    }

    free(a);
    free(b);
    free(c);
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
}


Overwriting q1_a.cu


In [22]:
!nvcc q1_a.cu -o output.out
!./output.out

Enter Number of Elements: 3

 Enter Array A: 1 2 4

 Enter Array B: 2 5 7
In Kernel function, i=0 In Kernel function, i=1 In Kernel function, i=2 
Final Array after adding both is : 3	7	11	

In [26]:
%%writefile q1_b.cu

#include <stdlib.h>
#include <stdio.h>
#include <cuda.h>

__global__ void vecAddKernel(int *A, int *B, int *C,int n){
    int i=blockDim.x*blockIdx.x+threadIdx.x;

    if(i<n){
        C[i]=A[i]+B[i];
    }
}

int main(){
    int n;
    printf("Enter Number of Elements: ");
    scanf("%d",&n);

    int *a,*b,*c;

    a=(int *)malloc(n*sizeof(int));
    b=(int *)malloc(n*sizeof(int));
    c=(int *)malloc(n*sizeof(int));

    printf("\n Enter Array A: ");

    for(int i=0;i<n;i++){
        scanf("%d", &a[i]);
    }

    printf("\n Enter Array B: ");

    for(int i=0;i<n;i++){
        scanf("%d", &b[i]);
    }

    int *da, *db, *dc;

    int size=n*sizeof(int);

    //part-1
    cudaMalloc((void **)&da,size);
    cudaMalloc((void **)&db,size);
    cudaMalloc((void **)&dc,size);

    cudaMemcpy(da, a, size,cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, size,cudaMemcpyHostToDevice);

    // 1 block and n threads
    vecAddKernel<<<1,n>>>(da,db,dc,n);

    //n blocks and 1 thread
    vecAddKernel<<<n,1>>>(da,db,dc,n);


    // Error check
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA error: %s\n", cudaGetErrorString(err));
    }

    // part-3 transfer to C

    cudaMemcpy(c,dc,size,cudaMemcpyDeviceToHost);

    printf("\nFinal Array after adding both is : ");

    for(int i=0;i<n;i++){
        printf("%d\t",c[i]);
    }

    free(a);
    free(b);
    free(c);
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
}

Overwriting q1_b.cu


In [27]:
!nvcc q1_b.cu -o output.out
!./output.out

Enter Number of Elements: 3

 Enter Array A: 5 6 7

 Enter Array B: 1 4 3

Final Array after adding both is : 6	10	10	

In [36]:
%%writefile q3.cu
#include<stdio.h>
#include<stdlib.h>
#include<cuda.h>

__global__ void ConvertToSines(double* a,int n){
    int i=blockDim.x*blockIdx.x+threadIdx.x;

    if(i<n){
       a[i]=sin(a[i]);
    }
}

int main(){
    double*a, *da;
    int n;
    printf("Enter Number of Elements :");
    scanf("%d",&n);

    int size=n*sizeof(double);
    a=(double*)malloc(size);

    printf("Enter Elements: ");
    for(int i=0;i<n;i++){
        scanf("%lf",&a[i]);
    }

    cudaMalloc((void**)&da,size);
    cudaMemcpy(da,a, size, cudaMemcpyHostToDevice);

    ConvertToSines<<<1,n>>>(da,n);

    cudaMemcpy(a,da,size,cudaMemcpyDeviceToHost);

    printf("\nSines of Elements are: ");
    for(int i=0;i<n;i++){
        printf("%lf\t",a[i]);
    }

    cudaFree(da);
    free(a);
}

Overwriting q3.cu


In [37]:
!nvcc q3.cu -o output.out
!./output.out

Enter Number of Elements :3
Enter Elements: 30 60 90

Sines of Elements are: -0.988032	-0.304811	0.893997	

In [48]:
%%writefile add_q1.cu
#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>

__global__ void calculateY(double* x, double*y,int m, int c, int n){

    int i=blockDim.x*blockIdx.x+threadIdx.x;

    if(i<n){
       y[i]=m*x[i]+c;
    }
}

int main(){
    int n;
    printf("Enter Number of Elements: ");
    scanf("%d",&n);

    double *x,*y;
    int m, c;


    x=(double *)malloc(n*sizeof(double));
    y=(double *)malloc(n*sizeof(double));

    printf("\n Enter Array X: ");

    for(int i=0;i<n;i++){
        scanf("%lf", &x[i]);
    }
    printf("Enter Slope and intercept: ");
    scanf("%d %d", &m, &c);

    double *dy,*dx;

    int size=n*sizeof(double);

    cudaMalloc((void **)&dy,size);
    cudaMalloc((void**)&dx,size);

    cudaMemcpy(dy, y, size,cudaMemcpyHostToDevice);
    cudaMemcpy(dx, x, size,cudaMemcpyHostToDevice);

    calculateY<<<1,n>>>(dx,dy,m,c,n);

    cudaMemcpy(y,dy,size,cudaMemcpyDeviceToHost);

    printf("\nFinal Array Y=mx+c i.e Y=(%d)*(x)+%d is : ",m,c);

    for(int i=0;i<n;i++){
        printf("%lf\t",y[i]);
    }

    free(x);
    free(y);

    cudaFree(dx);
    cudaFree(dy);

}

Overwriting add_q1.cu


In [49]:
!nvcc add_q1.cu -o output.out
!./output.out

Enter Number of Elements: 5

 Enter Array X: 7 3 4 6 3
Enter Slope and intercept: 3 2

Final Array Y=mx+c i.e Y=(3)*(x)+2 is : 23.000000	11.000000	14.000000	20.000000	11.000000	